In [ ]:
import os

import dotenv
import numpy as np
import pandas as pd


In [ ]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [ ]:
dotenv.load_dotenv(dotenv.find_dotenv())

## état du fichier

* des mauvaises lignes
* utf-8 mais des éléments non reconnus
* orienté structures
* des institutions publiques à priori
* pas de SIRET, de SIREN ou de code_insee
* contact plutôt bien renseigné

In [ ]:
df = pd.read_csv(
    os.environ["CD35_FILE_URL"],
    sep=";",
    encoding_errors="replace",
    on_bad_lines="warn",
    dtype=str,
)
df = df.replace(["", np.nan], None)
df.sample(5)


#### Nombre de lignes

In [ ]:
df.shape[0]

#### Taux de remplissage des champs

In [ ]:
def compute_field_occupancy_rates(df):
    return ((1 - df.isnull().sum() / df.shape[0]) * 100).sort_values(ascending=False)

In [ ]:
# taux de remplissage des champs
compute_field_occupancy_rates(df).to_frame()

## typologie de structure

via `ORG_SIGLE`

problèmes:
* présent à 69%
* beaucoup de valeurs cependant.

In [ ]:
# les 20 sigles les + fréquents
df.ORG_SIGLE.value_counts(dropna=False)[:20].to_frame()

In [ ]:
df.ORG_DESC.str.len().median()

### Départements

In [ ]:
df.ORG_CP.map(lambda x: str(int(x))[:2] if not pd.isna(x) else x).value_counts(dropna=False).to_frame()

In [ ]:
categories_flags_structures_df = df.apply(
    lambda row: {
        "pmi": ("maternelle" in row.ORG_NOM.lower() and "infantile" in row.ORG_NOM.lower()) or "pmi" in row.ORG_NOM.lower(),
        "ccas": pd.notnull(row.ORG_SIGLE) and "CCAS" == row.ORG_SIGLE,
        "saad": pd.notnull(row.ORG_SIGLE) and "SAAD" == row.ORG_SIGLE,
        "ehpad": pd.notnull(row.ORG_SIGLE) and "EHPAD" == row.ORG_SIGLE,
        "epiceries": "épiceries" in row.ORG_NOM.lower(),
        "admr": "admr" in row.ORG_NOM.lower(),
        "cmp": "cmpp" in row.ORG_NOM.lower() or ("centre" in row.ORG_NOM.lower() and "psycho" in row.ORG_NOM.lower()),
        "espace_jeux": "espace" in row.ORG_NOM.lower() and "jeu" in row.ORG_NOM.lower(),
        "halte_garderie": "halte-garderie" in row.ORG_NOM.lower(),
        "mairie": "mairie" in row.ORG_NOM.lower(),
        "pae": "pae" in row.ORG_NOM.lower(),
        "crèche": "crèche" in row.ORG_NOM.lower(),
        "sessad": "sessad" in row.ORG_NOM.lower(),
    },
    axis="columns",
    result_type="expand",
)

df[~categories_flags_structures_df.any(axis="columns")].sort_values("ORG_NOM").sample(10)

In [ ]:
categories_flags_structures_df.sum().sort_values(ascending=False).plot(
    kind="bar", grid=True, rot=35, figsize=(20, 8)
)